<a href="https://colab.research.google.com/github/olal4/Recognition-of-architectural-elements-on-potograph-of-building-facades/blob/main/Masked_Windows_Facade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tworzenie nowego zestawu z zamaskowanymi oknami i augumentacją

In [ ]:
import cv2
import numpy as np
import os
import json
import random
from albumentations import (Compose, RandomBrightnessContrast, Rotate, GaussianBlur, HueSaturationValue, BboxParams)
import matplotlib.pyplot as plt

# Funkcja do rysowania adnotacji na obrazie
def draw_annotations(image, annotations, color=(0, 255, 0)):
    for anno in annotations:
        x, y, w, h = map(int, anno['bbox'])
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    return image

# Funkcja do zasłaniania losowo wybranych okien
def mask_random_windows(image, annotations, mask_ratio=0.5):
    masked_annotations = []
    for anno in annotations:
        if anno['category_id'] == 3:  # numer kategorii "okno"
            if random.random() < mask_ratio:  # losowanie okno z prawdopodobieństwem mask_ratio
                x, y, w, h = map(int, anno['bbox'])
                # Obliczanie średniego koloru ściany wokół okna
                border = 5
                top = max(y - border, 0)
                bottom = min(y + h + border, image.shape[0])
                left = max(x - border, 0)
                right = min(x + w + border, image.shape[1])

                wall_colors = []
                if top < y:
                    wall_colors.append(np.mean(image[top:y, left:right], axis=(0, 1)))
                if y + h < bottom:
                    wall_colors.append(np.mean(image[y+h:bottom, left:right], axis=(0, 1)))
                if left < x:
                    wall_colors.append(np.mean(image[top:bottom, left:x], axis=(0, 1)))
                if x + w < right:
                    wall_colors.append(np.mean(image[top:bottom, x+w:right], axis=(0, 1)))

                if wall_colors:
                    color = np.mean(wall_colors, axis=0)  # średni kolor ściany
                    cv2.rectangle(image, (x, y), (x + w, y + h), color.tolist(), -1)
            else:
                masked_annotations.append(anno)  # Dodanie okna do nowych adnotacji jeśli nie zasłonięte
        else:
            masked_annotations.append(anno)  # Dodanie innych kategorie bez zmian
    return image, masked_annotations

# Funkcja do augmentacji obrazu i adnotacji
def augment_image_and_bboxes(image, bboxes):
    transform = Compose([
        RandomBrightnessContrast(p=0.5),
        Rotate(limit=20, p=0.5),
        GaussianBlur(p=0.3),
        HueSaturationValue(p=0.5)
    ], bbox_params=BboxParams(format='coco', label_fields=['category_ids']))

    transformed = transform(image=image, bboxes=bboxes, category_ids=[1]*len(bboxes))
    return transformed['image'], transformed['bboxes'], transform

# Funkcja do przetwarzania obrazu
def process_single_image(image_path, annotations, image_id, output_dir=None, mask_ratio=0.5):
    image = cv2.imread(image_path)
    image_annotations = [anno for anno in annotations if anno['image_id'] == image_id]

    # Zasłonięcie losowych okien
    masked_image, masked_annotations = mask_random_windows(image.copy(), image_annotations, mask_ratio)

    # Zapisanie obrazu z zasłoniętymi oknami
    if output_dir:
        os.makedirs(os.path.join(output_dir, 'img'), exist_ok=True)
        output_image_path = os.path.join(output_dir, 'img', os.path.basename(image_path))
        cv2.imwrite(output_image_path, masked_image)
        print(f"Obraz z zamaskowanymi oknami zapisano w {output_image_path}")

    # Augmentacja obrazu i adnotacji
    bboxes = [anno['bbox'] for anno in masked_annotations]
    augmented_image, augmented_bboxes, transform = augment_image_and_bboxes(masked_image, bboxes)

    # Zaktualizowane adnotacje
    augmented_annotations = []
    for i, bbox in enumerate(augmented_bboxes):
        new_anno = masked_annotations[i].copy()
        new_anno['bbox'] = bbox
        augmented_annotations.append(new_anno)

    # Zapisanie obrazu po augmentacji
    if output_dir:
        augmented_image_path = os.path.join(output_dir, 'img', 'aug_' + os.path.basename(image_path))
        cv2.imwrite(augmented_image_path, augmented_image)
        print(f"Obraz z augmentacją zapisano w {augmented_image_path}")

    return masked_annotations, augmented_annotations, os.path.basename(image_path), 'aug_' + os.path.basename(image_path)

# Ścieżki do katalogów i pliku z adnotacjami
input_dir = '/content/drive/MyDrive/ścieżka-do-obrazów-wejściowych'
annotations_file = '/content/drive/MyDrive/ścieżka-do-adnotacji-dla-danych-wejściowych'
output_dir = '/content/drive/MyDrive/ścież-do-folderu-dla-nowego-zestawu'

# Wczytanie adnotacji
with open(annotations_file) as f:
    annotations_data = json.load(f)

# Nowe katalogi wyjściowe
os.makedirs(os.path.join(output_dir, 'img'), exist_ok=True)

# Nowy słownik adnotacji
new_annotations = {"images": [], "annotations": [], "categories": annotations_data["categories"]}

# Przetwarzanie obrazów
for idx, image_info in enumerate(annotations_data['images']):
    print(f"Processing image {idx+1}/{len(annotations_data['images'])}: {image_info['file_name']}")
    image_path = os.path.join(input_dir, image_info['file_name'])
    masked_annotations, augmented_annotations, masked_image_file_name, augmented_image_file_name = process_single_image(image_path, annotations_data['annotations'], image_info['id'], output_dir=output_dir)

    new_image_id = len(new_annotations["images"])
    new_annotations["images"].append({"id": new_image_id, "file_name": masked_image_file_name})

    for anno in masked_annotations:
        new_annotations["annotations"].append({**anno, "image_id": new_image_id})

    new_image_id += 1
    new_annotations["images"].append({"id": new_image_id, "file_name": augmented_image_file_name})

    for anno in augmented_annotations:
        new_annotations["annotations"].append({**anno, "image_id": new_image_id})

# Zapisanie nowych adnotacji do pliku JSON po przetworzeniu wszystkich obrazów
new_annotations_file = os.path.join(output_dir, 'annotations.json')
with open(new_annotations_file, 'w') as f:
    json.dump(new_annotations, f)
print(f"Zaktualizowane adnotacje zapisano w {new_annotations_file}")

Podział zestawu danych na treningowy i walidacyjny

In [ ]:
!pip install pycocotools

In [ ]:
import json
import os
import random
from shutil import copyfile

input_annotations_path = '/content/drive/MyDrive/ścieżka-do-pliku-z-adnotacjami'
input_images_dir = '/content/drive/MyDrive/ścieżka-do-folderu-ze-zdjęciami'
train_output_dir = '/content/drive/MyDrive/ścieżka-do-folderu-na-zestaw-treningowy'
val_output_dir = '/content/drive/MyDrive/ścieżka-do-folderu-na-zestaw-walidacyjny'

with open(input_annotations_path, 'r') as f:
    data = json.load(f)

train_ratio = 0.8
val_ratio = 0.2

# Losowy podział danych
random.shuffle(data['images'])
train_images = data['images'][:int(len(data['images']) * train_ratio)]
val_images = data['images'][int(len(data['images']) * train_ratio):]

# Tworzenie mapowania image_id na nowe id
image_id_mapping = {}
for i, img in enumerate(train_images):
    image_id_mapping[img['id']] = i
for i, img in enumerate(val_images):
    image_id_mapping[img['id']] = i + len(train_images)

# Aktualizacja id obrazów
for i, img in enumerate(train_images):
    img['id'] = i
for i, img in enumerate(val_images):
    img['id'] = i + len(train_images)

# Aktualizacja adnotacji
train_annotations = []
val_annotations = []
for anno in data['annotations']:
    if anno['image_id'] in image_id_mapping:
        new_anno = anno.copy()
        new_anno['image_id'] = image_id_mapping[anno['image_id']]
        if new_anno['image_id'] < len(train_images):
            train_annotations.append(new_anno)
        else:
            val_annotations.append(new_anno)

# Tworzenie nowych katalogów na dane treningowe i walidacyjne
os.makedirs(os.path.join(train_output_dir, 'img'), exist_ok=True)
os.makedirs(os.path.join(val_output_dir, 'img'), exist_ok=True)

# Kopiowanie obrazów do odpowiednich katalogów
for img in train_images:
    copyfile(os.path.join(input_images_dir, img['file_name']), os.path.join(train_output_dir, 'img', img['file_name']))
for img in val_images:
    copyfile(os.path.join(input_images_dir, img['file_name']), os.path.join(val_output_dir, 'img', img['file_name']))

# Tworzenie plików JSON z adnotacjami dla nowych zestawów
train_data = {'images': train_images, 'annotations': train_annotations, 'categories': data['categories']}
val_data = {'images': val_images, 'annotations': val_annotations, 'categories': data['categories']}

with open(os.path.join(train_output_dir, 'annotations.json'), 'w') as f:
    json.dump(train_data, f)
with open(os.path.join(val_output_dir, 'annotations.json'), 'w') as f:
    json.dump(val_data, f)

print("Podział zestawu danych na treningowy i walidacyjny zakończony!")


Tworzenie unikalnych id

In [ ]:
import json

def update_ids(json_path, start_annotation_id=0, start_image_id=0):
    with open(json_path, 'r') as file:
        data = json.load(file)

    unique_annotation_ids = set()
    unique_image_ids = set()

    # Aktualizacja ID
    current_max_image_id = start_image_id
    image_id_mapping = {}
    for image in data['images']:
        old_id = image['id']
        current_max_image_id += 1
        image['id'] = current_max_image_id
        image_id_mapping[old_id] = current_max_image_id
        unique_image_ids.add(image['id'])

    # Sprawdzenie czy image_id w adnotacjach pokrywa się z obrazami
    for annotation in data['annotations']:
        if annotation['image_id'] not in image_id_mapping:
            print(f"Uwaga: Adnotacja {annotation['id']} zawiera nie prawidłowe image_id {annotation['image_id']}")

    # Aktualizacja adnotacji id w image_id
    current_max_annotation_id = start_annotation_id
    for annotation in data['annotations']:
        current_max_annotation_id += 1
        annotation['id'] = current_max_annotation_id
        annotation['image_id'] = image_id_mapping[annotation['image_id']]
        unique_annotation_ids.add(annotation['id'])

    with open(json_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"ID w pliku {json_path} zostały zaktualizowane poprawnie!")

train_json_path = '/content/drive/MyDrive/ścieżka-do-adnotacji-treningowych'
val_json_path = '/content/drive/MyDrive/ścieżka-do-adnotacji-walidacyjnych'

update_ids(train_json_path, start_annotation_id=0, start_image_id=0)
update_ids(val_json_path, start_annotation_id=1000000, start_image_id=1000000)

Dodanie rozmiarów obrazów do adnotacji

In [ ]:
import os
import json
from PIL import Image

input_annotations_path = '/content/drive/MyDrive/ścieżka-do-pliku-z-adnotacjami'
output_annotations_path = '/content/drive/MyDrive/ścieżka-dla-nowych-adnotacji'
images_directory = '/content/drive/MyDrive/ścieżka-do-obrazów'

with open(input_annotations_path, 'r') as f:
    new_data = json.load(f)

# Sprawdzenie rozmiarów obrazów i dodanie ich do nowych adnotacji
for img in new_data['images']:
    image_path = os.path.join(images_directory, img['file_name'])
    if os.path.exists(image_path):
        with Image.open(image_path) as image:
            width, height = image.size
            img['width'] = width
            img['height'] = height
    else:
        print(f"Uwaga: {image_path} nie istnieje.")

with open(output_annotations_path, 'w') as f:
    json.dump(new_data, f)

print(f"Zaktualizowane adnotacje zapisano w {output_annotations_path}")